In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
import os 
from scipy.sparse import linalg
from pathlib import Path
import itertools
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu"    

torch.set_default_dtype(torch.float64)
pi = torch.tensor(np.pi)
ZERO = torch.tensor([0.]).to(device)
torch.set_printoptions(precision=6)

class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1
    def evaluate_derivative(self, x, i):
        if self.k == 1:
            u1 = self.fc2(torch.heaviside(self.fc1(x),ZERO) * self.fc1.weight.t()[i-1:i,:] )
        else:
            u1 = self.fc2(self.k*F.relu(self.fc1(x))**(self.k-1) *self.fc1.weight.t()[i-1:i,:] )  
        return u1


In [6]:

def show_convergence_order(err_l2,exponent,dict_size,k,d, filename,write2file = False):
    
    if write2file:
        file_mode = "a" if os.path.exists(filename) else "w"
        f_write = open(filename, file_mode)
    
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    l2_order = -1/2-(2*k + 1)/(2*d)
    if write2file:
        f_write.write('dictionary size: {}\n'.format(dict_size))
        f_write.write("neuron num \t\t error \t\t order{} \t\t h10 error \\ order \n".format(l2_order))
    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            print("{} \t\t {:.6f} \t\t *  \n".format(neuron_nums[i],item ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t * \t\t \n".format(neuron_nums[i],item ))
        else: 
            print("{} \t\t {:.6f} \t\t {:.6f} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ))
    if write2file:     
        f_write.write("\n")
        f_write.close()


def show_convergence_order_latex(err_l2,exponent,k=1,d=1): 
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    l2_order = -1/2-(2*k + 1)/(2*d)
    print("neuron num  & \t $\\|u-u_n \\|_{{L^2}}$ & \t order $O(n^{{{:.2f}}})$  \\\\ \\hline \\hline ".format(l2_order))
    for i, item in enumerate(err_list):
        if i == 0: 
            print("{} \t\t & {:.6f} &\t\t *  \\\ \hline  \n".format(neuron_nums[i],item) )   
        else: 
            print("{} \t\t &  {:.2e} &  \t\t {:.2f} \\\ \hline  \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ) )


### 2D interface example   

### 2D $u(x_1,x_2) = \sqrt{x_1^2 + x_2^2}$ 

In [7]:
#N = 4096 
dim =2  
function_name = "cone2d"
for relu_k in [1,2]:  
    print("relu_k = {}".format(relu_k)) 
    trial_num = 5 
    errl2_trials = [] 
    for N0 in [2**12]: 
        s = 1 
        for trial in range(trial_num):
            exponent = 10
            num_epochs = 2**exponent  
            folder = './' 
            filename = folder + 'err_OGA_2D_{}_relu{}_neuron_{}_N_{}_randomized_trial{}.pt'.format(function_name,relu_k,num_epochs,s*N0,trial)
            errl2 = torch.load(filename)
            errl2_trials.append(errl2)
    errl2_trials = torch.stack(errl2_trials, dim = 0)
    errl2_ave = errl2_trials.mean(dim = 0)
    errl2_var = errl2_trials.var(dim = 0)

    show_convergence_order_latex(errl2_ave,exponent, k=relu_k, d=dim)


relu_k = 1
neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.25})$  \\ \hline \hline 
4 		 & 0.362867 &		 *  \\ \hline  

8 		 &  3.80e-02 &  		 3.26 \\ \hline  

16 		 &  1.47e-02 &  		 1.36 \\ \hline  

32 		 &  5.72e-03 &  		 1.37 \\ \hline  

64 		 &  1.94e-03 &  		 1.56 \\ \hline  

128 		 &  6.78e-04 &  		 1.51 \\ \hline  

256 		 &  2.28e-04 &  		 1.57 \\ \hline  

512 		 &  6.51e-05 &  		 1.80 \\ \hline  

1024 		 &  2.10e-05 &  		 1.64 \\ \hline  

relu_k = 2
neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.75})$  \\ \hline \hline 
4 		 & 0.357457 &		 *  \\ \hline  

8 		 &  9.22e-02 &  		 1.96 \\ \hline  

16 		 &  2.77e-02 &  		 1.73 \\ \hline  

32 		 &  9.34e-03 &  		 1.57 \\ \hline  

64 		 &  3.51e-03 &  		 1.41 \\ \hline  

128 		 &  1.42e-03 &  		 1.31 \\ \hline  

256 		 &  5.39e-04 &  		 1.39 \\ \hline  

512 		 &  2.01e-04 &  		 1.42 \\ \hline  

1024 		 &  7.83e-05 &  		 1.36 \\ \hline  



### 2D $L^2$ shaped example 